In [2]:
import numpy as np

import chugunov_indicator as chug
from chugunov_indicator.default_data import DefaultScreeningData

import pynucastro as pyna
import yt

In [3]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt17526/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-02 02:35:10,259 Parameters: current_time              = 0.800001438968278
yt : [INFO     ] 2024-10-02 02:35:10,260 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-02 02:35:10,261 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-02 02:35:10,262 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [4]:
reaclib_library = pyna.ReacLibLibrary()

nuclei = [field[2:-1] for field in np.array(ds.field_list)[:,1] if "X(" in field]
comp = pyna.Composition(nuclei)

mynet = reaclib_library.linking_nuclei(comp.keys())
pynet = pyna.PythonNetwork(libraries=[mynet])

screen_map = pyna.screening.get_screening_map(
    pynet.get_rates(),
    symmetric_screening=pynet.symmetric_screening
)

In [4]:
comp.set_solar_like()
plasma = pyna.make_plasma_state(1e8, 1e4, comp.get_molar())

In [6]:
pair = screen_map[5]
scn_fac = pyna.make_screen_factors(pair.n1, pair.n2)

In [11]:
%%timeit

F = pyna.screening.chugunov_2009(plasma, scn_fac)

1.76 µs ± 41.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
pyna.screening.chugunov_2009(plasma, scn_fac)

1.2989009131276164

In [9]:
screen_interp = DefaultScreeningData.default_interpolator

In [10]:
%%timeit

skip = chug.skip_screening(plasma, scn_fac, screen_interp, method="linear")

232 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


This means that for individual screening factor calculations, the current interpolator is about 100 times slower than simply computing the screening factor directly.

The bottleneck here is the performance of `RegularGridInterpolator.__call__`. Possible avenues of improvement include:
- Using an alternative interpolator. While `RegularGridInterpolator` is the best option I could find from `scipy`, there exist other packages with faster interpolators. It may also be possible to implement something directly if needed.
- Find numerical models for $c$'s dependence on any of $\bar{A}, \overline{Z^2}, Z_1, Z_2$. This could potentially reduce the dimensionality of the interpolator's inputs or perhaps even circumvent the need for one at all.
- Formulate an analytical model for $c$ by directly manipulating the equations in `chugunov_2009`. This is by far the hardest option given here, but it could be rewarding to try and figure out why the power law $\rho_b \propto T^3$ seems to hold here.